<a href="https://colab.research.google.com/github/aayushis1203/dietcheck/blob/main/00_DietCheck_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 00 – Data Collection & Preparation for DietCheck

**Course:** CS6120 – Natural Language Processing  
**Project:** DietCheck – NLP System for Dietary Claim Verification  
**Notebook:** `00` – Core data preparation, numeric labels for Task 1, and claim subsets for Task 2.

This notebook does the following:

1. Loads the **core product table** (`products.csv`) for DietCheck.
2. Computes **per-serving nutrition features** and **Task 1 dietary labels**:
   - `keto_compliant`, `high_protein`, `low_sodium`, `low_fat`  
     (using the FDA-style thresholds in the research plan).
3. Creates **train/validation/test splits** with label-combination awareness.
4. Extracts a **small, high-precision set of claim-like strings** from `products.csv`
   for **Task 2 manual annotation** → `candidate_claims_task2.csv`.
5. Builds a **claim-rich subset from OpenFoodFacts via HuggingFace** using
   `labels_tags` → `openfoodfacts_claims_subset.csv` for additional Task 2 data.

You should run this notebook top-to-bottom in a Colab or local environment with internet access
(for the HuggingFace step).


In [44]:
# ======================================================================
# Cell 1: Imports, paths, and logging
# ======================================================================

import os
import math
from pathlib import Path

import numpy as np
import pandas as pd

import logging

# Configure basic logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%H:%M:%S",
)
logger = logging.getLogger(__name__)

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# Data directory
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

print(f"DATA_DIR set to: {DATA_DIR.resolve()}")
print(f"Random seed set to: {RANDOM_SEED}")

DATA_DIR set to: /content/data
Random seed set to: 42


In [45]:
# ======================================================================
# Cell 2: Load products.csv
# ======================================================================

products_path = DATA_DIR / "products.csv"

if not products_path.exists():
    raise FileNotFoundError(
        f"Expected {products_path} to exist.\n"
        "Please copy your DietCheck products table to data/products.csv and re-run."
    )

df = pd.read_csv(products_path)
print(f"✓ Loaded products.csv with shape: {df.shape}")
print(f"✓ Columns: {list(df.columns)}")
print(f"✓ Product count: {len(df)}")

✓ Loaded products.csv with shape: (279, 29)
✓ Columns: ['product_id', 'name', 'brand', 'category', 'ingredients', 'serving_size_g', 'energy_100g', 'fat_100g', 'saturated_fat_100g', 'carbs_100g', 'fiber_100g', 'sugars_100g', 'protein_100g', 'sodium_100g', 'net_carbs_100g', 'energy_per_serving', 'fat_per_serving', 'saturated_fat_per_serving', 'carbs_per_serving', 'fiber_per_serving', 'sugars_per_serving', 'protein_per_serving', 'sodium_per_serving', 'net_carbs_per_serving', 'keto_compliant', 'high_protein', 'low_sodium', 'low_fat', 'label_combination']
✓ Product count: 279


In [46]:
# ======================================================================
# Cell 3: Compute per-serving nutrition and Task 1 labels
# ======================================================================

print("\n➤ Computing per-serving nutrition and Task 1 dietary labels...\n")

# Check if per-serving columns already exist
serving_cols = ["energy_per_serving", "fat_per_serving", "protein_per_serving",
                "sodium_per_serving", "carbs_per_serving", "fiber_per_serving",
                "saturated_fat_per_serving", "sugars_per_serving", "net_carbs_per_serving"]

if all(col in df.columns for col in serving_cols):
    print("✓ Per-serving columns already exist, skipping computation")
else:
    print("Computing per-serving values from 100g data...")

    # Compute per-serving values
    df["energy_per_serving"] = df["energy_100g"] * df["serving_size_g"] / 100.0
    df["fat_per_serving"] = df["fat_100g"] * df["serving_size_g"] / 100.0
    df["saturated_fat_per_serving"] = df["saturated_fat_100g"] * df["serving_size_g"] / 100.0
    df["carbs_per_serving"] = df["carbs_100g"] * df["serving_size_g"] / 100.0
    df["fiber_per_serving"] = df["fiber_100g"] * df["serving_size_g"] / 100.0
    df["sugars_per_serving"] = df["sugars_100g"] * df["serving_size_g"] / 100.0
    df["protein_per_serving"] = df["protein_100g"] * df["serving_size_g"] / 100.0
    df["sodium_per_serving"] = df["sodium_100g"] * df["serving_size_g"] / 100.0

    # Net carbs
    df["net_carbs_per_serving"] = (
        df["carbs_per_serving"] - df["fiber_per_serving"]
    ).fillna(df["carbs_per_serving"])

    print("✓ Per-serving values computed")

# Check if Task 1 labels already exist
label_cols = ["keto_compliant", "high_protein", "low_sodium", "low_fat"]

if all(col in df.columns for col in label_cols):
    print("✓ Task 1 labels already exist, skipping computation")
else:
    print("Computing Task 1 dietary classification labels...")

    # FDA thresholds (adjusted for keto as per research plan)
    KETO_NET_CARBS_THRESHOLD = 40.0  # Adjusted from 5g as documented
    HIGH_PROTEIN_THRESHOLD = 10.0     # grams (20% DV)
    LOW_SODIUM_THRESHOLD = 140.0      # mg
    LOW_FAT_THRESHOLD = 3.0           # grams

    df["keto_compliant"] = (df["net_carbs_per_serving"] <= KETO_NET_CARBS_THRESHOLD).fillna(False)
    df["high_protein"] = (df["protein_per_serving"] >= HIGH_PROTEIN_THRESHOLD).fillna(False)
    df["low_sodium"] = (df["sodium_per_serving"] <= LOW_SODIUM_THRESHOLD).fillna(False)
    df["low_fat"] = (df["fat_per_serving"] <= LOW_FAT_THRESHOLD).fillna(False)

    print("✓ Task 1 labels computed")

# Create label combination for stratification
df["label_combination"] = (
    df["keto_compliant"].astype(str) + "_" +
    df["high_protein"].astype(str) + "_" +
    df["low_sodium"].astype(str) + "_" +
    df["low_fat"].astype(str)
)

print(f"\n✓ Total products: {len(df)}")
print(f"✓ Unique label combinations: {df['label_combination'].nunique()}")


➤ Computing per-serving nutrition and Task 1 dietary labels...

✓ Per-serving columns already exist, skipping computation
✓ Task 1 labels already exist, skipping computation

✓ Total products: 279
✓ Unique label combinations: 15


In [47]:
# ======================================================================
# Cell 4: Data quality and stratification report
# ======================================================================

print("\n" + "="*70)
print("DATA QUALITY REPORT")
print("="*70)

# Overall counts
print(f"\n📊 OVERALL STATISTICS")
print(f"  • Total products: {len(df)}")
print(f"  • Products with ingredients: {df['ingredients'].notna().sum()}")
print(f"  • Products with complete nutrition: {df[['fat_per_serving', 'protein_per_serving', 'sodium_per_serving', 'carbs_per_serving']].notna().all(axis=1).sum()}")

# Task 1 label distribution
print(f"\n🏷️  TASK 1 LABEL DISTRIBUTION")
for label in ["keto_compliant", "high_protein", "low_sodium", "low_fat"]:
    count = df[label].sum()
    pct = (count / len(df) * 100) if len(df) > 0 else 0
    print(f"  • {label}: {count} ({pct:.1f}%)")

# Category distribution (if exists)
if "category" in df.columns:
    print(f"\n📦 CATEGORY DISTRIBUTION (Top 8)")
    print(df["category"].value_counts().head(8))

# Label combinations
print(f"\n🔀 LABEL COMBINATIONS (Top 10)")
print(df["label_combination"].value_counts().head(10))

# Nutrient completeness
print(f"\n🧪 NUTRIENT DATA COMPLETENESS")
nutrient_cols = ["fat_per_serving", "protein_per_serving", "sodium_per_serving",
                 "carbs_per_serving", "fiber_per_serving", "sugars_per_serving"]
for col in nutrient_cols:
    if col in df.columns:
        count = df[col].notna().sum()
        pct = (count / len(df) * 100) if len(df) > 0 else 0
        print(f"  • {col}: {count}/{len(df)} ({pct:.1f}%)")

print("\n" + "="*70)


DATA QUALITY REPORT

📊 OVERALL STATISTICS
  • Total products: 279
  • Products with ingredients: 279
  • Products with complete nutrition: 279

🏷️  TASK 1 LABEL DISTRIBUTION
  • keto_compliant: 90 (32.3%)
  • high_protein: 105 (37.6%)
  • low_sodium: 124 (44.4%)
  • low_fat: 103 (36.9%)

📦 CATEGORY DISTRIBUTION (Top 8)
category
en:plant-based-foods-and-beverages         130
en:dairies                                  47
en:beverages-and-beverages-preparations     29
en:condiments                               23
en:snacks                                   18
en:seafood                                  10
en:meals                                     6
en:meats-and-their-products                  5
Name: count, dtype: int64

🔀 LABEL COMBINATIONS (Top 10)
label_combination
0_1_0_0    68
1_0_1_1    40
0_0_0_0    30
0_0_1_0    29
0_0_1_1    21
1_0_0_0    15
0_0_0_1    14
1_0_1_0    13
0_1_1_0    12
1_0_0_1    12
Name: count, dtype: int64

🧪 NUTRIENT DATA COMPLETENESS
  • fat_per_serving: 2

In [48]:
# ======================================================================
# Cell 5: Create train/validation/test splits (robust to rare combos)
# ======================================================================

from sklearn.model_selection import train_test_split

print("\n➤ Creating train/validation/test splits...\n")

if "label_combination" not in df.columns:
    raise KeyError("Expected 'label_combination' column to exist before creating splits.")

# ------------------------------------------------------------------
# 1) Handle rare label combinations (appearing only once in full df)
# ------------------------------------------------------------------
combo_counts = df["label_combination"].value_counts()

# Combinations we can safely stratify on (at least 2 samples in full df)
combos_ge2 = combo_counts[combo_counts >= 2].index

df_strat = df[df["label_combination"].isin(combos_ge2)].copy()
df_rare = df[~df["label_combination"].isin(combos_ge2)].copy()

print(f"  • Total products: {len(df)}")
print(f"  • Stratifiable products (combo count ≥ 2): {len(df_strat)}")
print(f"  • Rare combos (count = 1): {len(df_rare)}")

# If we basically have no stratifiable data, fall back to simple split
if len(df_strat) < 2:
    print("\n⚠️  Too few stratifiable products. Falling back to NON-stratified split.\n")

    df_train, df_temp = train_test_split(
        df,
        test_size=0.3,
        random_state=RANDOM_SEED,
        shuffle=True,
    )

    df_val, df_test = train_test_split(
        df_temp,
        test_size=0.5,
        random_state=RANDOM_SEED,
        shuffle=True,
    )

else:
    # ------------------------------------------------------------------
    # 2) First split: TRAIN vs (VAL+TEST) with stratification
    # ------------------------------------------------------------------
    print("\n➤ Performing stratified split on frequent label combinations (train vs temp)...\n")

    df_train_strat, df_temp = train_test_split(
        df_strat,
        test_size=0.3,
        random_state=RANDOM_SEED,
        stratify=df_strat["label_combination"],
    )

    # ------------------------------------------------------------------
    # 3) Second split: VAL vs TEST
    #    Try stratification, but only if every class in df_temp has ≥ 2
    # ------------------------------------------------------------------
    temp_combo_counts = df_temp["label_combination"].value_counts()
    min_temp_count = temp_combo_counts.min()

    if min_temp_count < 2:
        print("⚠️  Cannot stratify VAL/TEST: some label_combinations in temp have only 1 sample.")
        print("   → Using NON-stratified split for VAL/TEST.\n")

        df_val, df_test = train_test_split(
            df_temp,
            test_size=0.5,
            random_state=RANDOM_SEED,
            shuffle=True,
        )
    else:
        print("➤ Performing stratified split for VAL/TEST...\n")

        df_val, df_test = train_test_split(
            df_temp,
            test_size=0.5,
            random_state=RANDOM_SEED,
            stratify=df_temp["label_combination"],
        )

    # ------------------------------------------------------------------
    # 4) Add all rare-combo products into the TRAIN set
    # ------------------------------------------------------------------
    if not df_rare.empty:
        print(f"➤ Adding {len(df_rare)} rare-combo products to the TRAIN set only")
        df_train = pd.concat([df_train_strat, df_rare], ignore_index=True)
    else:
        df_train = df_train_strat.copy()

# ------------------------------------------------------------------
# 5) Final sanity prints
# ------------------------------------------------------------------
print(f"\n✓ Train set: {len(df_train)} products ({len(df_train)/len(df)*100:.1f}%)")
print(f"✓ Validation set: {len(df_val)} products ({len(df_val)/len(df)*100:.1f}%)")
print(f"✓ Test set: {len(df_test)} products ({len(df_test)/len(df)*100:.1f}%)")

print("\nLabel combination counts by split (top 10):")
print("\n  TRAIN:")
print(df_train["label_combination"].value_counts().head(10))
print("\n  VAL:")
print(df_val["label_combination"].value_counts().head(10))
print("\n  TEST:")
print(df_test["label_combination"].value_counts().head(10))

# ------------------------------------------------------------------
# 6) Save splits
# ------------------------------------------------------------------
df_train.to_csv(DATA_DIR / "train.csv", index=False)
df_val.to_csv(DATA_DIR / "val.csv", index=False)
df_test.to_csv(DATA_DIR / "test.csv", index=False)

print(f"\n✓ Saved splits to:")
print(f"  • {DATA_DIR / 'train.csv'}")
print(f"  • {DATA_DIR / 'val.csv'}")
print(f"  • {DATA_DIR / 'test.csv'}")



➤ Creating train/validation/test splits...

  • Total products: 279
  • Stratifiable products (combo count ≥ 2): 278
  • Rare combos (count = 1): 1

➤ Performing stratified split on frequent label combinations (train vs temp)...

⚠️  Cannot stratify VAL/TEST: some label_combinations in temp have only 1 sample.
   → Using NON-stratified split for VAL/TEST.

➤ Adding 1 rare-combo products to the TRAIN set only

✓ Train set: 195 products (69.9%)
✓ Validation set: 42 products (15.1%)
✓ Test set: 42 products (15.1%)

Label combination counts by split (top 10):

  TRAIN:
label_combination
0_1_0_0    47
1_0_1_1    28
0_0_0_0    21
0_0_1_0    20
0_0_1_1    15
1_0_0_0    11
0_0_0_1    10
1_0_1_0     9
0_1_1_0     8
1_0_0_1     8
Name: count, dtype: int64

  VAL:
label_combination
0_1_0_0    12
0_0_1_0     6
0_0_0_0     5
1_0_0_1     3
1_0_1_1     3
0_1_1_0     2
1_0_1_0     2
0_0_0_1     2
1_0_0_0     2
0_1_1_1     2
Name: count, dtype: int64

  TEST:
label_combination
0_1_0_0    9
1_0_1_1    

In [49]:
# ======================================================================
# Cell 6: Candidate claim extraction (HELPER for manual Task 2 annotation)
# ======================================================================
# NOTE: This cell ONLY extracts candidate text snippets.
# All Task 2 labels (claim_verifiable, claim_conflict, explanations)
# must be manually annotated by humans.
# ======================================================================

import re

print("\n➤ Extracting claim-like strings for MANUAL Task 2 annotation\n")
print("⚠️  This is a HELPER step - no automatic labels are created.")
print("⚠️  All claim_verifiable and claim_conflict labels must be done manually.\n")

source_df = df_train.copy()
print(f"  ⮕ Using TRAINING split: {len(source_df)} products")

TEXT_FIELDS = [f for f in ["name", "category", "brand"] if f in source_df.columns]
print(f"  ⮕ Scanning text fields: {TEXT_FIELDS}\n")

CLAIM_PATTERNS = {
    "low_sugar": [
        r"\bno\s+added\s+sugar\b",
        r"\bwithout\s+added\s+sugar\b",
        r"\bsugar[-\s]?free\b",
    ],
    "low_fat": [
        r"\blow[-\s]?fat\b",
        r"\b0\s*%\s*fat\b",
        r"\bfat[-\s]?free\b",
    ],
    "high_protein": [
        r"\b(high|rich)\s+in\s+protein\b",
        r"\bprotein[-\s]?rich\b",
        r"\bsource\s+of\s+protein\b",
    ],
    "high_fiber": [
        r"\b(high|rich)\s+in\s+fib(re|er)s?\b",
        r"\bsource\s+of\s+fib(re|er)s?\b",
    ],
    "low_sodium": [
        r"\blow\s+(salt|sodium)\b",
        r"\breduced\s+salt\b",
        r"\breduced\s+sodium\b",
        r"\bno\s+added\s+salt\b",
    ],
    "gluten_free": [
        r"\bgluten[-\s]?free\b",
    ],
    "lactose_free": [
        r"\blactose[-\s]?free\b",
    ],
    "keto": [
        r"\bketo(?:genic)?\b",
        r"\bketo[-\s]?friendly\b",
    ],
    "light": [
        r"\blight\b",
        r"\blightly\s+salted\b",
    ],
}

compiled_patterns = {
    k: [re.compile(p, flags=re.IGNORECASE) for p in v]
    for k, v in CLAIM_PATTERNS.items()
}

def extract_claims_from_text(pid, field_name, text, context_window=25):
    if not isinstance(text, str) or not text.strip():
        return []
    candidates = []
    for claim_type, regex_list in compiled_patterns.items():
        for regex in regex_list:
            for match in regex.finditer(text):
                start, end = match.span()
                left = max(0, start - context_window)
                right = min(len(text), end + context_window)
                snippet = text[left:right].strip()
                candidates.append(
                    {
                        "product_id": pid,
                        "claim_text": snippet,
                        "claim_type_hint": claim_type,
                        "source_field": field_name,
                        "full_text": text,
                    }
                )
    return candidates

all_candidates = []
for _, row in source_df.iterrows():
    pid = row.get("product_id", None)
    for field in TEXT_FIELDS:
        text = row.get(field, None)
        all_candidates.extend(
            extract_claims_from_text(pid, field, text, context_window=25)
        )

if not all_candidates:
    print("⚠️  No candidate claims found with current patterns.")
    candidates_df = pd.DataFrame(
        columns=["product_id", "claim_text", "claim_type_hint", "source_field", "full_text"]
    )
else:
    candidates_df = pd.DataFrame(all_candidates)
    candidates_df = candidates_df.drop_duplicates(
        subset=["product_id", "claim_text", "claim_type_hint", "source_field"]
    ).reset_index(drop=True)

print(f"  ⮕ Extracted {len(candidates_df)} claim-like strings")

if not candidates_df.empty:
    print("\n  ⮕ Claim type hints (for manual review):")
    print(candidates_df["claim_type_hint"].value_counts())
    print(f"\n  ⮕ Products with ≥1 claim snippet: {candidates_df['product_id'].nunique()}")

claims_path = DATA_DIR / "candidate_claims_task2.csv"
candidates_df.to_csv(claims_path, index=False)
print(f"\n✓ Saved candidate claim strings to: {claims_path}")
print("  → Use this CSV as a STARTING POINT for manual Task 2 annotation")


➤ Extracting claim-like strings for MANUAL Task 2 annotation

⚠️  This is a HELPER step - no automatic labels are created.
⚠️  All claim_verifiable and claim_conflict labels must be done manually.

  ⮕ Using TRAINING split: 195 products
  ⮕ Scanning text fields: ['name', 'category', 'brand']

  ⮕ Extracted 4 claim-like strings

  ⮕ Claim type hints (for manual review):
claim_type_hint
low_fat    3
light      1
Name: count, dtype: int64

  ⮕ Products with ≥1 claim snippet: 2

✓ Saved candidate claim strings to: data/candidate_claims_task2.csv
  → Use this CSV as a STARTING POINT for manual Task 2 annotation


In [54]:
# ======================================================================
# Cell 7: Load claim-rich OFF subset from HuggingFace
# ======================================================================

from datasets import load_dataset

print("\n➤ Loading OpenFoodFacts claim-rich subset...\n")

# Target claim tags from OFF
TARGET_TAGS = [
    "en:no-gluten",
    "en:gluten-free",
    "en:vegan",
    "en:vegetarian",
    "en:organic",
    "en:no-lactose",
    "en:lactose-free",
    "en:no-palm-oil",
    "en:palm-oil-free",
    "en:low-fat",
    "en:low-sugar",
    "en:no-added-sugar",
    "en:high-protein",
    "en:high-fiber",
]

# Map OFF tags to our claim types
CLAIM_LABEL_MAP = {
    "en:no-gluten": "gluten_free",
    "en:gluten-free": "gluten_free",
    "en:vegan": "vegan",
    "en:vegetarian": "vegetarian",
    "en:organic": "organic",
    "en:no-lactose": "lactose_free",
    "en:lactose-free": "lactose_free",
    "en:no-palm-oil": "palm_oil_free",
    "en:palm-oil-free": "palm_oil_free",
    "en:low-fat": "low_fat",
    "en:low-sugar": "low_sugar",
    "en:no-added-sugar": "no_added_sugar",
    "en:high-protein": "high_protein",
    "en:high-fiber": "high_fiber",
}

print(f"Mapped labels: {len(CLAIM_LABEL_MAP)} unique tags\n")

def extract_main_text(val):
    """Extract main text from multilingual dict or return string."""
    if isinstance(val, dict):
        return val.get("en", "") or val.get("en-us", "") or ""
    return str(val) if val else ""

def extract_ingredients_text(val):
    """Extract ingredient text from multilingual dict."""
    if isinstance(val, dict):
        return val.get("en", "") or val.get("en-us", "") or ""
    return str(val) if val else ""

def get_float(d, key):
    """Extract float from nutriments dict, return None if invalid."""
    if not isinstance(d, dict):
        return None
    val = d.get(key)
    if val is None:
        return None
    try:
        f = float(val)
        return f if not (math.isnan(f) or math.isinf(f)) else None
    except (ValueError, TypeError):
        return None

def extract_nutriments(nutriments):
    """Extract key nutrients as floats (or None)."""
    if not isinstance(nutriments, dict):
        return {
            "energy_100g": None,
            "fat_100g": None,
            "saturated_fat_100g": None,
            "carbs_100g": None,
            "fiber_100g": None,
            "sugars_100g": None,
            "protein_100g": None,
            "sodium_100g": None,
        }

    return {
        "energy_100g": get_float(nutriments, "energy-kcal_100g"),
        "fat_100g": get_float(nutriments, "fat_100g"),
        "saturated_fat_100g": get_float(nutriments, "saturated-fat_100g"),
        "carbs_100g": get_float(nutriments, "carbohydrates_100g"),
        "fiber_100g": get_float(nutriments, "fiber_100g"),
        "sugars_100g": get_float(nutriments, "sugars_100g"),
        "protein_100g": get_float(nutriments, "proteins_100g"),
        "sodium_100g": get_float(nutriments, "sodium_100g"),
    }

MAX_ROWS = 2000

print("Loading OpenFoodFacts dataset (streaming from HuggingFace)...")
ds = load_dataset("openfoodfacts/product-database", split="food", streaming=True)

rows = []
seen_codes = set()
n_scanned = 0

for example in ds:
    n_scanned += 1
    labels_tags = example.get("labels_tags") or []
    labels_tags = [t for t in labels_tags if isinstance(t, str)]
    matching_tags = [t for t in labels_tags if t in TARGET_TAGS]
    if not matching_tags:
        continue

    code = example.get("code")
    if not code or code in seen_codes:
        continue
    seen_codes.add(code)

    claim_types = []
    for t in matching_tags:
        mapped = CLAIM_LABEL_MAP.get(t)
        if mapped:
            claim_types.append(mapped)
    if not claim_types:
        continue

    product_name = extract_main_text(example.get("product_name"))
    brand = (example.get("brands") or "").strip()
    categories = (example.get("categories") or "").strip()
    ingredients_text = extract_ingredients_text(example.get("ingredients_text"))
    labels_str = (example.get("labels") or "").strip()
    nutriments = extract_nutriments(example.get("nutriments"))

    row = {
        "product_id": code,
        "name": product_name,
        "brand": brand,
        "category": categories,
        "ingredients_text": ingredients_text,
        "labels": labels_str,
        "labels_tags": "|".join(labels_tags),
        "claim_type_hint": ";".join(sorted(set(claim_types))),
        "source_field": "labels/labels_tags",
        "full_text": " | ".join(
            [x for x in [product_name, brand, categories, ingredients_text, labels_str] if x]
        ),
        **nutriments,
    }
    rows.append(row)

    if len(rows) % 200 == 0:
        print(f"  ⮕ Collected {len(rows)} claim-rich products (scanned {n_scanned})...")

    if len(rows) >= MAX_ROWS:
        break

print(f"\n✓ Finished. Collected {len(rows)} claim-rich products (scanned {n_scanned} total rows).")

df_claims = pd.DataFrame(rows)
hf_output_path = DATA_DIR / "openfoodfacts_claims_subset.csv"
df_claims.to_csv(hf_output_path, index=False)
print(f"✓ Saved claim-rich subset to: {hf_output_path}")

# Show summary
print(f"\n📊 OFF Claim Subset Summary:")
print(f"  • Total products: {len(df_claims)}")
print(f"  • With nutrition data: {df_claims[['fat_100g', 'protein_100g', 'sodium_100g', 'carbs_100g']].notna().any(axis=1).sum()}")
print(f"  • With ingredients: {df_claims['ingredients_text'].notna().sum()}")


➤ Loading OpenFoodFacts claim-rich subset...

Mapped labels: 14 unique tags

Loading OpenFoodFacts dataset (streaming from HuggingFace)...
  ⮕ Collected 200 claim-rich products (scanned 3090)...
  ⮕ Collected 400 claim-rich products (scanned 6062)...
  ⮕ Collected 600 claim-rich products (scanned 10910)...
  ⮕ Collected 800 claim-rich products (scanned 12581)...
  ⮕ Collected 1000 claim-rich products (scanned 14069)...
  ⮕ Collected 1200 claim-rich products (scanned 16534)...
  ⮕ Collected 1400 claim-rich products (scanned 20310)...
  ⮕ Collected 1600 claim-rich products (scanned 21411)...
  ⮕ Collected 1800 claim-rich products (scanned 23181)...
  ⮕ Collected 2000 claim-rich products (scanned 25987)...

✓ Finished. Collected 2000 claim-rich products (scanned 25987 total rows).
✓ Saved claim-rich subset to: data/openfoodfacts_claims_subset.csv

📊 OFF Claim Subset Summary:
  • Total products: 2000
  • With nutrition data: 0
  • With ingredients: 2000


In [55]:
# ======================================================================
# Cell 8: Filter noisy OFF rows (RELAXED - no nutrient requirement)
# ======================================================================

print("\n➤ Filtering OFF products for quality...\n")

# Try to load if not in memory
try:
    df_claims
except NameError:
    df_claims = pd.read_csv(DATA_DIR / "openfoodfacts_claims_subset.csv")
    print(f"Loaded {len(df_claims)} rows from saved CSV")

nutr_cols = [
    "energy_100g", "fat_100g", "saturated_fat_100g",
    "carbs_100g", "fiber_100g", "sugars_100g",
    "protein_100g", "sodium_100g",
]

# 1) Basic non-empty checks
mask_text = df_claims["full_text"].fillna("").str.len() > 10
mask_claim = df_claims["claim_type_hint"].fillna("").str.strip().ne("")

df_tmp = df_claims[mask_text & mask_claim].copy()
print(f"  ⮕ After text + claim filter: {len(df_tmp)}")

# 2) Nutrient filter – DISABLED for OFF claim pool
# We accept products even if all nutrients are missing, because
# for Task 2 we mainly care about claim text.
print("  ⮕ Skipping nutrient-based filtering (nutriments often missing in OFF subset)")

# 3) Remove obviously bad categories
bad_keywords = [
    "pet food", "dog", "cat", "beauty", "cosmetics",
    "shampoo", "soap", "supplements", "dietary supplement",
]

if len(df_tmp) > 0:
    cat = df_tmp["category"].fillna("").str.lower()
    mask_bad_cat = cat.apply(lambda x: any(k in x for k in bad_keywords))
    df_tmp = df_tmp[~mask_bad_cat].copy()
    print(f"  ⮕ After category filter: {len(df_tmp)}")

# 4) Deduplicate by product_id
df_filtered = df_tmp.drop_duplicates(subset="product_id").reset_index(drop=True)

print(f"\n✓ Final filtered OFF pool: {len(df_filtered)} products")

if len(df_filtered) > 0:
    # These will likely be 0 because nutriments are often missing, but we log anyway
    with_nutr = df_filtered[nutr_cols].notna().any(axis=1).sum()
    print(f"  • With ANY nutrition data: {with_nutr}")
    print(f"  • With ingredients: {df_filtered['ingredients_text'].notna().sum()}")

    off_filtered_path = DATA_DIR / "openfoodfacts_claims_filtered.csv"
    df_filtered.to_csv(off_filtered_path, index=False)
    print(f"\n✓ Saved filtered pool to: {off_filtered_path}")
else:
    print("⚠️  No products passed filtering. Consider relaxing criteria further.")



➤ Filtering OFF products for quality...

  ⮕ After text + claim filter: 2000
  ⮕ Skipping nutrient-based filtering (nutriments often missing in OFF subset)
  ⮕ After category filter: 1996

✓ Final filtered OFF pool: 1996 products
  • With ANY nutrition data: 0
  • With ingredients: 1996

✓ Saved filtered pool to: data/openfoodfacts_claims_filtered.csv


In [56]:
# ======================================================================
# Cell 9: Sample OFF products for manual annotation (FIXED)
# ======================================================================

print("\n➤ Creating stratified OFF sample for manual annotation...\n")

if len(df_filtered) == 0:
    print("⚠️  No products available for sampling.")
    print("   Skipping OFF sampling step.")
    sampled = pd.DataFrame()
else:
    # Create primary_claim from claim_type_hint
    df_filtered["primary_claim"] = (
        df_filtered["claim_type_hint"]
        .fillna("")
        .str.split(";")
        .str[0]
        .str.strip()
    )

    # Remove rows where primary_claim is empty
    df_filtered = df_filtered[df_filtered["primary_claim"].ne("")].copy()

    print(f"  ⮕ Unique primary claims: {df_filtered['primary_claim'].nunique()}")

    # Target number of OFF products
    TARGET_OFF = 120

    per_label_target = max(1, TARGET_OFF // max(1, df_filtered["primary_claim"].nunique()))

    def _sample_group(g):
        n = min(len(g), per_label_target)
        return g.sample(n=n, random_state=RANDOM_SEED)

    sampled = (
        df_filtered
        .groupby("primary_claim", group_keys=False)
        .apply(_sample_group)
    )

    # Top up if under target
    if len(sampled) < TARGET_OFF:
        remaining = df_filtered.drop(sampled.index)
        extra_needed = TARGET_OFF - len(sampled)
        if extra_needed > 0 and len(remaining) > 0:
            extra = remaining.sample(
                n=min(extra_needed, len(remaining)),
                random_state=RANDOM_SEED,
            )
            sampled = pd.concat([sampled, extra], ignore_index=True)

    sampled = sampled.reset_index(drop=True)

    print(f"\n✓ Final OFF sample size: {len(sampled)}")
    print("\n  Claim distribution:")
    print(sampled["primary_claim"].value_counts())

    # Save sampled subset
    off_sampled_path = DATA_DIR / "openfoodfacts_off_sample_for_manual.csv"
    sampled.to_csv(off_sampled_path, index=False)
    print(f"\n✓ Saved OFF sample to: {off_sampled_path}")


➤ Creating stratified OFF sample for manual annotation...

  ⮕ Unique primary claims: 9

✓ Final OFF sample size: 120

  Claim distribution:
primary_claim
gluten_free       24
organic           24
vegan             14
vegetarian        14
lactose_free      13
low_fat           13
no_added_sugar    13
low_sugar          3
palm_oil_free      2
Name: count, dtype: int64

✓ Saved OFF sample to: data/openfoodfacts_off_sample_for_manual.csv


/tmp/ipython-input-492357703.py:38: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_sample_group)


In [57]:
# ======================================================================
# NEW Cell: Build OFF claim table (per-claim rows) for manual Task 2
# ======================================================================

import re

print("\n➤ Building OFF claim table for manual Task 2 annotation...\n")

if 'sampled' not in globals():
    try:
        sampled = pd.read_csv(DATA_DIR / "openfoodfacts_off_sample_for_manual.csv")
        print(f"Loaded OFF sample from CSV with {len(sampled)} rows")
    except FileNotFoundError:
        print("⚠️ OFF sample not found. Run Cell 9 first.")
        sampled = pd.DataFrame()

if sampled.empty:
    print("⚠️ OFF sample is empty. Skipping claim-table creation.")
else:
    off_df = sampled.copy()
    print(f"  ⮕ Using OFF sample with {len(off_df)} products")

    def split_labels(labels_str):
        if not isinstance(labels_str, str):
            return []
        parts = re.split(r"[;,|]", labels_str)
        return [p.strip() for p in parts if p and p.strip()]

    KEYWORDS = [
        "free", "no ", "without", "low", "reduced", "rich",
        "source", "high", "light", "0%", "0 %", "sans", "faible", "pauv",
    ]

    def looks_like_claim(text):
        if not text:
            return False
        t = text.lower()
        if len(t) < 4:
            return False
        return any(k in t for k in KEYWORDS)

    rows = []

    for _, row in off_df.iterrows():
        pid = row.get("product_id", None)
        name = row.get("name", "")
        brand = row.get("brand", "")
        full_text = row.get("full_text", "")

        labels = row.get("labels", "")
        label_candidates = split_labels(labels)

        text = full_text if isinstance(full_text, str) else ""
        extra_candidates = []
        for chunk in re.split(r"[.!?/]", text):
            chunk = chunk.strip()
            if looks_like_claim(chunk):
                extra_candidates.append(chunk)

        all_candidates = set(
            c for c in (label_candidates + extra_candidates) if looks_like_claim(c)
        )

        if not all_candidates:
            # still create row so you can mark "no claim"
            rows.append({
                "product_id": pid,
                "name": name,
                "brand": brand,
                "source": "openfoodfacts_off_sample",
                "raw_claim_text": "",
                "full_text": full_text,
            })
        else:
            for claim_text in all_candidates:
                rows.append({
                    "product_id": pid,
                    "name": name,
                    "brand": brand,
                    "source": "openfoodfacts_off_sample",
                    "raw_claim_text": claim_text,
                    "full_text": full_text,
                })

    off_claims_df = pd.DataFrame(rows).drop_duplicates(
        subset=["product_id", "raw_claim_text", "source"]
    ).reset_index(drop=True)

    print(f"  ⮕ Built {len(off_claims_df)} OFF claim rows")
    print(f"  ⮕ Products covered: {off_claims_df['product_id'].nunique()}")

    claims_out_path = DATA_DIR / "claims_off_for_manual_task2.csv"
    off_claims_df.to_csv(claims_out_path, index=False)

    print(f"\n✓ Saved OFF claim table for manual annotation to: {claims_out_path}")
    display(off_claims_df.head(10))



➤ Building OFF claim table for manual Task 2 annotation...

  ⮕ Using OFF sample with 120 products
  ⮕ Built 305 OFF claim rows
  ⮕ Products covered: 120

✓ Saved OFF claim table for manual annotation to: data/claims_off_for_manual_task2.csv


,product_id,name,brand,source,raw_claim_text,full_text
0,0027331000431,"[{'lang': 'main', 'text': 'Corn Tortillas'}, {...",Ole Mexican Foods Inc.,openfoodfacts_off_sample,,"[{'lang': 'main', 'text': 'Corn Tortillas'}, {..."
1,0016000457232,"[{'lang': 'main', 'text': 'PEANUT BUTTER DARK ...",Nature Valley,openfoodfacts_off_sample,"'}] | Sin gluten,Sin aromas artificiales,Sin c...","[{'lang': 'main', 'text': 'PEANUT BUTTER DARK ..."
2,0016000457232,"[{'lang': 'main', 'text': 'PEANUT BUTTER DARK ...",Nature Valley,openfoodfacts_off_sample,en:No high fructose corn syrup,"[{'lang': 'main', 'text': 'PEANUT BUTTER DARK ..."
3,0028000255664,"[{'lang': 'main', 'text': ""Taster's Choice 100...",Nescafé,openfoodfacts_off_sample,"[{'lang': 'main', 'text': ""Taster's Choice 100...","[{'lang': 'main', 'text': ""Taster's Choice 100..."
4,0028000255664,"[{'lang': 'main', 'text': ""Taster's Choice 100...",Nescafé,openfoodfacts_off_sample,Sans gluten,"[{'lang': 'main', 'text': ""Taster's Choice 100..."
5,0025317007214,"[{'lang': 'main', 'text': 'Uncured Beef Corn D...",Applegate Naturals,openfoodfacts_off_sample,No gluten,"[{'lang': 'main', 'text': 'Uncured Beef Corn D..."
6,0025317007214,"[{'lang': 'main', 'text': 'Uncured Beef Corn D...",Applegate Naturals,openfoodfacts_off_sample,"GLUTEN-FREE BATTER: YELLOW CORN FLOUR, SUGAR, ...","[{'lang': 'main', 'text': 'Uncured Beef Corn D..."
7,0025317007214,"[{'lang': 'main', 'text': 'Uncured Beef Corn D...",Applegate Naturals,openfoodfacts_off_sample,'}] | No gluten,"[{'lang': 'main', 'text': 'Uncured Beef Corn D..."
8,0025317007214,"[{'lang': 'main', 'text': 'Uncured Beef Corn D...",Applegate Naturals,openfoodfacts_off_sample,"GLUTEN-FREE BATTER: YELLOW CORN FLOUR, SUGAR, ...","[{'lang': 'main', 'text': 'Uncured Beef Corn D..."
9,0025317007214,"[{'lang': 'main', 'text': 'Uncured Beef Corn D...",Applegate Naturals,openfoodfacts_off_sample,CONTAINS LESS THAN 2% OF THE FOLLOWING: SEA SA...,"[{'lang': 'main', 'text': 'Uncured Beef Corn D..."


In [58]:
# ======================================================================
# Cell 10: Final summary and reproducibility check
# ======================================================================

print("\n" + "="*70)
print("NOTEBOOK 00 COMPLETION SUMMARY")
print("="*70)

print("\n✅ COMPLETED TASKS:")
print("  1. Loaded base products.csv")
print("  2. Computed per-serving nutrition and Task 1 labels")
print("  3. Created stratified train/val/test splits")
print("  4. Generated candidate claims for manual Task 2 annotation")
print("  5. Loaded and filtered OpenFoodFacts claim-rich subset")
print("  6. Created stratified OFF sample for manual annotation")

print("\n📁 OUTPUT FILES:")
files_created = [
    "train.csv",
    "val.csv",
    "test.csv",
    "candidate_claims_task2.csv",
    "openfoodfacts_claims_subset.csv",
    "openfoodfacts_claims_filtered.csv",
    "openfoodfacts_off_sample_for_manual.csv"
]

for fname in files_created:
    path = DATA_DIR / fname
    if path.exists():
        size = path.stat().st_size / 1024
        print(f"  ✓ {fname} ({size:.1f} KB)")
    else:
        print(f"  ⚠️  {fname} (NOT FOUND)")

print("\n📊 CURRENT DATA STATUS:")
print(f"  • Products in train/val/test: {len(df)}")
print(f"  • Task 1 labels: ✓ Complete")
print(f"  • Task 2 claim candidates: {len(candidates_df)} snippets")
print(f"  • Task 3 ready: ✓ Ingredients available")

print("\n⏭️  NEXT STEPS FOR GRADE CONTRACT:")
print("\n  FOR B GRADE (minimum 120 products):")
print("    • Current products: {len(df)}")
if len(df) >= 120:
    print("    ✓ You have enough products!")
else:
    print(f"    ⚠️  Need {120 - len(df)} more products")
print("    • Manually annotate Task 1 on all products")
print("    • Manually annotate Task 2 on ≥50 claims")
print("    • Implement rule-based + TF-IDF baselines")

print("\n  FOR B+ GRADE (minimum 180 products):")
if len(df) >= 180:
    print("    ✓ You have enough products!")
else:
    print(f"    ⚠️  Need {180 - len(df)} more products")
print("    • Expand Task 2 to ≥120 claims with explanations")
print("    • Double-annotate ≥25 products (Cohen's kappa)")
print("    • Implement BERT baseline")

print("\n  FOR A- GRADE (minimum 200 products):")
if len(df) >= 200:
    print("    ✓ You have enough products!")
else:
    print(f"    ⚠️  Need {200 - len(df)} more products")
print("    • Expand Task 2 to ≥160 claims")
print("    • Start Task 3 BIO tagging on ≥80 products")
print("    • Implement multimodal model")

print("\n  FOR A GRADE (minimum 200 products):")
if len(df) >= 200:
    print("    ✓ You have enough products!")
else:
    print(f"    ⚠️  Need {200 - len(df)} more products")
print("    • Complete Task 3 on ≥120 products")
print("    • Implement claim-table model")
print("    • Perform slice-based analysis")

print("\n🔒 REPRODUCIBILITY:")
print(f"  • Random seed: {RANDOM_SEED}")
print("  • All splits saved with stratification")
print("  • All intermediate files saved")

print("\n⚠️  IMPORTANT REMINDERS:")
print("  • candidate_claims_task2.csv is ONLY a helper")
print("  • ALL Task 2 labels must be manually annotated")
print("  • Task 3 BIO tagging must be manual")
print("  • Follow FDA thresholds for Task 1 verification")

print("\n" + "="*70)
print("Notebook 00 complete! Ready for manual annotation.")
print("="*70)


NOTEBOOK 00 COMPLETION SUMMARY

✅ COMPLETED TASKS:
  1. Loaded base products.csv
  2. Computed per-serving nutrition and Task 1 labels
  3. Created stratified train/val/test splits
  4. Generated candidate claims for manual Task 2 annotation
  5. Loaded and filtered OpenFoodFacts claim-rich subset
  6. Created stratified OFF sample for manual annotation

📁 OUTPUT FILES:
  ✓ train.csv (79.8 KB)
  ✓ val.csv (19.0 KB)
  ✓ test.csv (17.9 KB)
  ✓ candidate_claims_task2.csv (0.5 KB)
  ✓ openfoodfacts_claims_subset.csv (3491.5 KB)
  ✓ openfoodfacts_claims_filtered.csv (3478.7 KB)
  ✓ openfoodfacts_off_sample_for_manual.csv (244.0 KB)

📊 CURRENT DATA STATUS:
  • Products in train/val/test: 279
  • Task 1 labels: ✓ Complete
  • Task 2 claim candidates: 4 snippets
  • Task 3 ready: ✓ Ingredients available

⏭️  NEXT STEPS FOR GRADE CONTRACT:

  FOR B GRADE (minimum 120 products):
    • Current products: {len(df)}
    ✓ You have enough products!
    • Manually annotate Task 1 on all products
    